This notebook is meant to be a self-contained Python demonstration of how VTech typically runs ctrl-VQE simulations.

Unfortunately, I don't typically use either Jupyter or Python, so it may or may not be helpful. `^_^`

In particular, this notebook will use `ctrlq`, an old Python package written before my time (though I have fixed a couple bugs).
It is not as extensible or as efficient as my current Julia code, but it should be perfectly sufficient for matching against a first proof-of-principle experiment.

Be sure to build the C++ part of `ctrlq` according to the instructions on GitHub,
    and then make sure the `ctrlq` module is in the PYTHONPATH when you start the jupyter server,
    or else this notebook won't know what it's supposed to do!

For reference, GitHub versions of the Python `ctrlq` and the more modern Julia `CtrlVQE.jl` are available here:
- https://github.com/kmsherbertvt/ctrlq
- https://github.com/kmsherbertvt/CtrlVQE.jl

#### Specify the Chemistry

At present, our simulations' target observable is input directly as a dense matrix.

The `ctrlq` library has some old `qiskit` code that used to generate these matrices,
    but it is severely out of date and nearly impossible to get to work.

My own workflow is to use `pyscf` to find the Hartree-Fock orbitals and calculate integrals,
    then use `openfermion` to convert the integrals into one- and two-body tensors,
    map those onto qubit operators (usually via the parity mapping),
    then apply qubit tapering.
The result is a qubit operator (ie. sum of Paulis).

In practice, one would partition this operator into commuting groups
    and perform a distinct circuit to measure expectation values of each one.
    but for my simulations I just convert the full operator into a matrix,
    and I save it as a numpy `.npy` file.

So all my simulations ever have to do is load that `.npy` file...

##### Import numpy

In [1]:
import numpy
numpy.set_printoptions(3, suppress=True)

##### Load a molecular Hamiltonian

In [2]:
matrixfile = "matrix/H2_1.5.npy"
H_mol = numpy.load(matrixfile)
H_mol

array([[-0.661+0.j,  0.   +0.j,  0.   +0.j,  0.23 +0.j],
       [ 0.   +0.j, -0.911+0.j,  0.23 +0.j,  0.   +0.j],
       [ 0.   +0.j,  0.23 +0.j, -0.394+0.j,  0.   +0.j],
       [ 0.23 +0.j,  0.   +0.j,  0.   +0.j, -0.661+0.j]])

In [3]:
# The matrices in this package were generated from qiskit,
#    but openfermion generates a permuted matrix.
# As it turns out, the openfermion permutation works rather better for H2.
# You can turn the qiskit shape into the openfermion shape with the following permutation.
P = numpy.array([[0, 0, -1, 0], [-1, 0, 0, 0], [0, -1, 0, 0], [0, 0, 0, 1]])
H_mol = P @ H_mol @ P.conj().T
H_mol

array([[-0.394+0.j,  0.   +0.j,  0.23 +0.j,  0.   +0.j],
       [ 0.   +0.j, -0.661+0.j,  0.   +0.j, -0.23 +0.j],
       [ 0.23 +0.j,  0.   +0.j, -0.911+0.j,  0.   +0.j],
       [ 0.   +0.j, -0.23 +0.j,  0.   +0.j, -0.661+0.j]])

##### Infer the problem size

In [4]:
N = len(H_mol)
n = int(numpy.log2(N))

The only other thing to specify is the reference state on which the variational control pulse acts.

Typically, we have been using the Hartree-Fock state, which is some computational basis state
    (which one depends on exactly how the matrix was constructed),
    and we've just been implicitly assuming it can be prepared perfectly.

In principle, we could just always start out with `|0..0⟩`, the usual for hardware.
But the evolution time is bound to be longer,
    and starting far from a good reference makes us vulnerable to barren plateaus,
    so for now let's just do what we've done so far.

##### Specify the reference state

In [5]:
initket = [0]*n
initket[0] = 1

#### Specify the quantum computer

Oddly enough, the `ctrlq` library code I got my hands on only ever uses one device parameterization,
    based loosely on an IBMQ device four years ago.
It was clearly designed to accommodate something more general.
It just, uh, doesn't, and I never bothered to fix it.

It does, at the very least, let you feed in a device's static Hamiltonian as a dense matrix.

So I'm just basically going to copy/paste the code that it uses to compute the static Hamiltonian
    of the "baked in" device,
    and you can change the parameters of the device as needed.

Unfortunately, that device has linear couplings,
    so you will need to adjust the static Hamiltonian if you want an arbitrary graph.
    
(Alternatively, I could write you a notebook in Julia, using my rather-more-flexible code! `^_^`)

##### Define the device parameters

In [6]:
pi2 = 2 * numpy.pi
class DemoDevice:
    def __init__(self):
        self.w =[   # RESONANCE FREQUENCIES (radians per nanosecond)
            pi2 * 4.808049015463495,
            pi2 * 4.833254817254613,
            pi2 * 4.940051121317842,
            pi2 * 4.795960998582043,
        ]
        self.eta = [ # ANHARMONICITIES (radians per nanosecond)
            pi2* 0.3101773613134229,
            pi2* 0.2916170385725456,
            pi2* 0.3301773613134229,
            pi2* 0.2616170385725456,
        ]
        self.g = [   # COUPLING STRENGTHS (radians per nanosecond) [Order: 0-1, 1-2, 2-3, 3-0]
            pi2 *0.018312874435769682,
            pi2 *0.021312874435769682,
            pi2 *0.019312874435769682,
            pi2 *0.020312874435769682,
        ]

##### Construct the static Hamiltonian

In [7]:
from ctrlq import cvqe
from ctrlq.cvqe.omisc import cbas_, qbas_, anih, create, initial_state
def static_ham(nstate, nqubit = 2):

    diag_n = numpy.arange(nstate)
    diag_n = numpy.diagflat(diag_n)
    eye_n = numpy.eye(nstate, dtype=numpy.float64)
    diag_eye = 0.5 * numpy.dot(diag_n, diag_n - eye_n)
    astate = anih(nstate)
    cstate = create(nstate)

    dp = DemoDevice()
    
    ham_ = 0.0
    iwork = True
    for i in range(nqubit):

        h_ = dp.w[i]*diag_n - dp.eta[i]*diag_eye

        if not i:
            tmp_ = h_
            tmp_i = astate
        else:
            tmp_ = eye_n
            if i == nqubit-1:
                tmp_i = cstate
            else:
                tmp_i = eye_n
        
        for j in range(1,nqubit):
            if j == i:
                wrk = h_
                wrk_i = astate
            elif j == i+1:
                wrk = eye_n
                wrk_i = cstate
            else:
                wrk = eye_n
                wrk_i = eye_n
                
            tmp_ = numpy.kron(tmp_,wrk)
            if iwork:
                tmp_i = numpy.kron(tmp_i,wrk_i)

        ham_ += tmp_
        if iwork:
            tmp_i += tmp_i.conj().T
            tmp_i *= dp.g[i]
            ham_ += tmp_i
            
            if nqubit == 2:
                iwork = False
    
    return ham_

Now we can construct the transmon object used to simulate time evolution in a quantum computer.

We can choose to simulate `m` levels in each transmon;
    most pedagogical treatments simply assume `m=2` so the Hamiltonian can be written with Pauli operators.
There is an implicit assumption that leakage never happens,
    and that is tenuously enforced by only ever applying well-studied pulse shapes.
    
Admittedly, most of my own simulations in the last year do this too. `^_^`

But, let's *not* do that here...

##### Specify the size of the physical transmon space

In [8]:
m = 3

##### Construct the transmon object

In [9]:
H0 = static_ham(m, nqubit=n)
device = cvqe.transmon(
    nqubit=n,
    nstate=m,
    mham=H_mol,     # Why should the observable be a parameter to the device, you wonder? Me too...
    istate=initket,
    Hstatic=H0,
)

#### Specify the pulse

The `ctrlq` code implements square pulses and, I think, Gaussian pulses,
    though I've never tested the latter.

Actually, the only difficult in using different pulse shapes
    is in working out efficient analytic expressions for the gradient,
    needed for fast optimizations *in silico*.
This is very easy to generalize, and my Julia code does so,
    but I don't think it's easy with `ctrlq` as it stands. `^_^`

*In quanta*, the analytical gradient is probably more trouble than it's worth,
    at least for a first proof-of-concept,
    so I think you could really use whatever pulse shapes are easiest to implement on your device,
    but for comparison's sake I suppose square pulses are as good a place to start as any.

The biggest thing `ctrlq` lacks is any control on the phase of the pulse:
    pulse amplitudes are explicitly real,
    and it turns out the gradient expression is a bit different if you want to make them complex,
    so you can't just tweak things to accept a different number type.
    
The other major thing is pulse constraints: `ctrlq` does have them,
    some even more sophisticated than I have in my Julia code right now.
But it does not, uh, *document* them, and I haven't taken the time to figuring out how they work.

The example below explicilty shows a default for a parameter which I assume imposes simple box constraints on the drive strength,
    but my personal recommendation for a first proof-of-concept is to work with long-enough pulses
    that the optimizer never bothers to suggest pulse amplitudes you aren't comfortable applying.

##### Specify some meta-parameters

In [10]:
T = 12.0 # ns  # TOTAL PULSE DURATION
W = 4          # NUMBER OF DISTINCT WINDOWS

##### Construct the pulse

In [11]:
pulse = cvqe.pulse(
    nqubit=n, duration=T,
    shape='square', nwindow=W,
    amp_bound = 0.02, # For whatever reason, this seems to be a frequency rather than angular frequency like the others.
)

Pulse parameters for the `ctrlq` square pulse include:
- the drive frequencies for each qubit (assumed constant throughout the pulse duration)
- the strength for each window (negative numbers are allowed, for an implicit discrete switch to phase ϕ=π)
- the times at which windows switch (perhaps different for each qubit)

The switching times are not varied by the optimizer.
(At least not for the square pulse shape - perhaps they are for Gaussians?
They certainly could be... You could do it for the square pulse too,
    but for the infinitesimal width of a Dirac-delta function.
Too small for even the most precise of finite differences... `^_^`).

All these values are initialized to random numbers when the pulse is constructed,
    but I will now overwrite them with somewhat sensible fixed values so you can see their formatting.

##### Set the initial pulse parameters

In [12]:
pulse.freq = list(DemoDevice().w)[:n]  # Start on resonance of first n qubits.
pulse.amp = [
    [0.0]*W, # The list of amplitudes for each window on the first qubit.
    [0.0]*W, # The list of amplitudes for each window on the first qubit.
]
pulse.tseq = [
    [(1+i)*T/W for i in range(W)],  # Uniformly spaced windows. Note the first window always starts at 0.0, and the last ends at T.
    [(1+i)*T/W for i in range(W)],  # (Same thing for the second qubit in this example, but they needn't match!)
]

#### Run the optimization

In [13]:
r = int(200*T)  # Number of Trotter steps
ctrl = cvqe.control(pulse, device, nstep=r, solver='trotter', iprint=1)
E, lk = ctrl.optimize(
    method='l-bfgs-b',    # Actually the only option...
    normalize=True,       
    maxiter=100,           
    maxls=20,              
    gtol=1.0e-09,
    ftol=1.0e-09,
    gradient='numerical',
    shape='square',
)


  Pulse optimization ends
  CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  ------------------------------------------

  Printing progress
  --------------------------------------------------------
  --------------------------------------------------------
  Iter       Energy(H)      Leak(%)  Ediff(H)    Gnorm
    0     -0.910873550000   0.0000  0.0000e+00  0.0000e+00
    1     -0.932425345770   0.0264  2.1552e-02  9.5252e-01
    2     -0.948698861020   0.0209  1.6274e-02  6.0157e-01
    3     -0.967037751701   0.0859  1.8339e-02  5.6237e-01
    4     -0.975162218071   0.1042  8.1245e-03  4.8016e-01
    5     -0.984677892128   0.1014  9.5157e-03  3.5303e-01
    6     -0.988876815290   0.0877  4.1989e-03  2.6763e-01
    7     -0.990468405534   0.0790  1.5916e-03  1.4312e-01
    8     -0.993226201919   0.0829  2.7578e-03  1.2999e-01
    9     -0.995027443864   0.1449  1.8012e-03  9.3539e-02
   10     -0.995874900485   0.3119  8.4746e-04  1.7370e-01
   11     -0.996543438540   0.4296 

#### Check the results

In [14]:
print(f"ctrl-VQE found energy: {E} Ha")

E0, U0 = numpy.linalg.eigh(H_mol)
print(f"     Error with exact: {E-E0[0]}")

print(f"% loss due to leakage: {100*lk} %")

ctrl-VQE found energy: -0.9981493516714971 Ha
     Error with exact: 1.9047763366586423e-11
% loss due to leakage: 0.8724247755610093 %
